# Salary Prediction from LinkedIn Job Postings - Train XGBoost Model

In [1]:
import xgboost as xgb
import pandas as pd, numpy as np
import salary
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer

/Users/nginyc/repos/job_posting_salary_prediction/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Train & Evaluate Model

In [2]:
(X_train, y_train) = salary.get_train_dataset()
(X_test, y_test) = salary.get_test_dataset()

### Target normal salaries

In [3]:
model = make_pipeline(
    ColumnTransformer(
        transformers=[
            ('target_encoder', TargetEncoder(), ['norm_title', 'location_state', 'company_industries']),
            ('experience_level', salary.experience_level_encoder, ['formatted_experience_level']),
            ('work_type', salary.work_type_encoder, ['formatted_work_type']),
            ('remote_allowed', 'passthrough', ['remote_allowed']),
            ('company_employee_count', SimpleImputer(strategy='median'), ['company_employee_count']),
        ],
        remainder='drop'
    ),
    StandardScaler(),
    xgb.XGBRegressor(
        n_estimators=1000,
        max_depth=5,
        learning_rate=0.1,
        verbosity=2,
        objective='reg:squarederror',
        subsample=0.8,
        random_state=42
    )
).fit(X_train, y_train)

salary.evaluate_train_predictions(model.predict(X_train))

[01:03:08] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (27885, 7, 195195).
Train R2: 0.6729
Train RMSE: 35183.3770
Train MAE: 22961.5632


In [4]:
salary.evaluate_test_predictions(model.predict(X_test))

Test R2: 0.4875
Test RMSE: 41733.9019
Test MAE: 27586.5894
On average, our predicted salaries are $27586.59 off the true salaries
This is 36.47% better than a naive global mean


### Target log salaries


In [5]:
model_log = make_pipeline(
    ColumnTransformer(
        transformers=[
            ('target_encoder', TargetEncoder(), ['norm_title', 'location_state', 'company_industries']),
            ('experience_level', salary.experience_level_encoder, ['formatted_experience_level']),
            ('work_type', salary.work_type_encoder, ['formatted_work_type']),
            ('remote_allowed', 'passthrough', ['remote_allowed']),
            ('company_employee_count', SimpleImputer(strategy='median'), ['company_employee_count']),
        ],
        remainder='drop'
    ),
    StandardScaler(),
    TransformedTargetRegressor(
        regressor=xgb.XGBRegressor(
            n_estimators=1000,
            max_depth=5,
            learning_rate=0.1,
            verbosity=2,
            objective='reg:squarederror',
            subsample=0.8,
            random_state=42
        ),
        func=np.log10,
        inverse_func=lambda x: 10**x
    )
).fit(X_train, y_train)

salary.evaluate_train_predictions(model.predict(X_train))

[01:03:16] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (27885, 7, 195195).
Train R2: 0.6729
Train RMSE: 35183.3770
Train MAE: 22961.5632


In [6]:
salary.evaluate_test_predictions(model_log.predict(X_test))

Test R2: 0.4850
Test RMSE: 41835.5979
Test MAE: 26199.9455
On average, our predicted salaries are $26199.95 off the true salaries
This is 39.66% better than a naive global mean


### Feature Importance

In [7]:
feature_names = model[:-1].get_feature_names_out(salary.df_X.columns)

def get_important_features(coef, n=10):
    idxs = np.flip(np.argsort(np.abs(coef))[-n:])
    return pd.DataFrame([feature_names[idxs], coef[idxs]], index=["Feature", "Coefficient"]).transpose()

get_important_features(model[-1].feature_importances_)


,Feature,Coefficient
0,experience_level__formatted_experience_level,0.305902
1,target_encoder__norm_title,0.182569
2,target_encoder__company_industries,0.151451
3,target_encoder__location_state,0.101761
4,work_type__formatted_work_type,0.093816
5,company_employee_count__company_employee_count,0.086131
6,remote_allowed__remote_allowed,0.078371
